In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForCausalLM
import torch.nn as nn
import pandas as pd
import numpy as np
import math
import pickle
from sklearn.metrics import roc_auc_score
import nlpaug.augmenter.word as naw
from functools import partial
from functions import *
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
batch_size = 1
num_workers = 16
accumulation_steps = 64
verbose = 2048
lr = 8e-5
clip = 8e-3

save_model_path = get_next_folder_name('Model/')
if not os.path.exists(save_model_path):
    os.makedirs(save_model_path)
device = 'cuda'

In [3]:
## get random hyper-parameters ##
TARGET_MODEL = "mistralai/Mistral-7B-v0.1"
pred_type = np.random.choice(['LM','classification'])
config_type = np.random.choice(['prefix','prompt_encoder','prompt_txt','LoRA'],p=[0.3,0.3,0.1,0.3])
epochs = 2
alpha = np.random.rand()*0.1
aug_kwargs = dict(aug_max=np.random.randint(5,30),aug_p=np.random.rand()*0.3)
config_class,config_kwargs = get_random_config(config_type,pred_type,TARGET_MODEL)

In [4]:
TARGET_MODEL, pred_type, config_type, epochs, alpha, aug_kwargs, config_class, config_kwargs

('mistralai/Mistral-7B-v0.1',
 'classification',
 'prompt_encoder',
 2,
 0.017380905301284978,
 {'aug_max': 29, 'aug_p': 0.14823071313383032},
 peft.tuners.p_tuning.config.PromptEncoderConfig,
 {'task_type': 'SEQ_CLS',
  'num_virtual_tokens': 24,
  'encoder_hidden_size': 2048,
  'encoder_dropout': 0.10530677543381284,
  'encoder_num_layers': 1,
  'encoder_reparameterization_type': 'MLP'})

In [5]:
# load hyper-parameter for inference
# with open(save_model_path+'/config.pkl', 'rb') as pickle_file:
#     config = pickle.load(pickle_file)
# TARGET_MODEL, pred_type, config_type, epochs, alpha, aug_kwargs, config_class, config_kwargs = load_config(config)

Data

In [6]:
val = pd.read_csv('data/val_data.csv')
train = pd.read_csv('data/train_data.csv')

In [7]:
tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
# http://paraphrase.org/#/download
# https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb
aug = naw.SynonymAug(aug_src='ppdb',model_path='Model/ppdb-2.0-s-all',
                     aug_min=1,
                     **aug_kwargs)
# print(aug.augment(train.text.iloc[3]))

In [9]:
if pred_type == 'LM':
    prompt = tokenizer.batch_encode_plus(['Is this essay AI-generated, yes or no?'],add_special_tokens=False,return_tensors='pt')
    prompt,prompt_mask = prompt['input_ids'],prompt['attention_mask']
else:
    prompt,prompt_mask = None, None
train_data = TxtData(train,aug.augment)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, \
                        collate_fn=partial(collate_fn,tokenizer=tokenizer,prompt=prompt,prompt_mask=prompt_mask))
val_data = InfData(val)
val_loader = DataLoader(val_data, batch_size=batch_size*2, shuffle=False, num_workers=num_workers, \
                        collate_fn=partial(collate_inf,tokenizer=tokenizer,prompt=prompt,prompt_mask=prompt_mask))
# input_ids,attention_mask, label, score = next(iter(train_loader))

Model

In [10]:
from torch.nn.utils import clip_grad_value_
from transformers import BitsAndBytesConfig
from peft import get_peft_model

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
base_class = AutoModelForCausalLM if pred_type == 'LM' else AutoModelForSequenceClassification
base_model = base_class.from_pretrained(TARGET_MODEL,quantization_config=nf4_config, \
                                                          device_map={"":0},use_flash_attention_2=True)
peft_config = config_class(**config_kwargs)    
model = get_peft_model(base_model, peft_config)
model.config.pad_token_id = tokenizer.pad_token_id
# model.print_trainable_parameters()
trainable_params = [param for param in model.parameters() if param.requires_grad]
trainable_names = [name for name,param in model.named_parameters() if param.requires_grad]
# optimizer = torch.optim.AdamW(trainable_params,lr = lr,amsgrad=True,weight_decay=6e-3)
optimizer = torch.optim.SGD(trainable_params,lr=lr)
if pred_type == 'LM':
    model_lm = LM(model,tokenizer,config_kwargs.get('num_virtual_tokens', 0),alpha,config_type)
else:
    model_lm = Classification(model,alpha)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zhenlan/anaconda3/lib/python3.10/site-packages/peft/tuners/p_tuning/model.py:106: UserWarning: for MLP, the argument `encoder_num_layers` is ignored. Exactly 2 MLP layers are used.
  warnings.warn(


In [11]:
loss_fn = nn.BCEWithLogitsLoss()
loss_fct = torch.nn.CrossEntropyLoss()
best_auc = 0
for epoch in range(epochs):
    model.train()
    train_loss = 0
    skip = 0
    for i, (input_ids,attention_mask, label, score) in enumerate(train_loader):
        # train
        input_ids,attention_mask, label, score = input_ids.to('cuda'),attention_mask.to('cuda'), label.to('cuda'), score.to('cuda')
        loss = model_lm.get_loss(input_ids,attention_mask, label, score)
        if math.isinf(loss.item()) or math.isnan(loss.item()):
            skip += 1
            continue

        loss.backward()
        train_loss += loss.item()
        # print(i,train_loss)
        if (i + 1) % accumulation_steps == 0:
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()

        # eval    
        if (i + 1) % verbose == 0:
            model.eval()
            train_loss /= (verbose-skip)
            yhat = []
            for input_ids,attention_mask in val_loader:
                input_ids,attention_mask = input_ids.to('cuda'),attention_mask.to('cuda')
                out = model_lm.predict(input_ids,attention_mask).detach().cpu().numpy()
                yhat.append(out)
            yhat = np.concatenate(yhat)
            auc = roc_auc_score(val.label.to_numpy(), yhat)
            print(f"epoch {epoch} iter {i}: train loss {train_loss}, test AUC {auc}")
            if auc > best_auc:
                best_auc = auc
                torch.save({k: v for k, v in model.state_dict().items() if k in trainable_names}, save_model_path+'/weights.pth')
            train_loss = 0
            skip = 0
            model.train()

epoch 0 iter 2047: train loss 1.8627260276896322, test AUC 0.52608974524764
epoch 0 iter 4095: train loss 1.110802516244263, test AUC 0.5671491012543645
epoch 0 iter 6143: train loss 0.7810555421343679, test AUC 0.6288431397905082
epoch 0 iter 8191: train loss 0.7372587505351476, test AUC 0.6475623949308159
epoch 0 iter 10239: train loss 0.658708295615213, test AUC 0.7030901331953964
epoch 0 iter 12287: train loss 0.6330276468474416, test AUC 0.7340457778352516
epoch 0 iter 14335: train loss 0.5119002051502832, test AUC 0.7287738264580369
epoch 0 iter 16383: train loss 0.5003882889196234, test AUC 0.8190259924996767
epoch 0 iter 18431: train loss 0.4128437285005049, test AUC 0.8221037113668692
epoch 0 iter 20479: train loss 0.39672157420218923, test AUC 0.8554667011509117
epoch 0 iter 22527: train loss 0.4260173111860013, test AUC 0.8377245570929781
epoch 0 iter 24575: train loss 0.35463347213314833, test AUC 0.8787502909608171
epoch 0 iter 26623: train loss 0.3591720194772279, test AU

KeyboardInterrupt: 

In [ ]:
# save config and local score
config = save_config(TARGET_MODEL, pred_type, config_type, epochs, alpha, aug_kwargs, config_kwargs)
config['local auc'] = best_auc
with open(save_model_path+'/config.pkl', 'wb') as pickle_file:
    pickle.dump(config, pickle_file)

Inference

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForCausalLM
# import torch.nn as nn
# import pandas as pd
# import numpy as np
# import math
# import pickle
# from sklearn.metrics import roc_auc_score
# import nlpaug.augmenter.word as naw
# from functools import partial
# from functions import *
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# batch_size = 1
# num_workers = 16
# accumulation_steps = 64
# verbose = 2048
# lr = 8e-5
# clip = 8e-3

# save_model_path = 'Model/model03'
# device = 'cuda'
# # load hyper-parameter for inference
# with open(save_model_path+'/config.pkl', 'rb') as pickle_file:
#     config = pickle.load(pickle_file)
# TARGET_MODEL, pred_type, config_type, epochs, alpha, aug_kwargs, config_class, config_kwargs = load_config(config)

# val = pd.read_csv('data/val_data.csv')
# tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL)
# tokenizer.pad_token = tokenizer.eos_token
# if pred_type == 'LM':
#     prompt = tokenizer.batch_encode_plus(['Is this essay AI-generated, yes or no?'],add_special_tokens=False,return_tensors='pt')
#     prompt,prompt_mask = prompt['input_ids'],prompt['attention_mask']
# else:
#     prompt,prompt_mask = None, None
# val_data = InfData(val)
# val_loader = DataLoader(val_data, batch_size=batch_size*4, shuffle=False, num_workers=num_workers, \
#                         collate_fn=partial(collate_inf,tokenizer=tokenizer,prompt=prompt,prompt_mask=prompt_mask))
# # input_ids,attention_mask, label, score = next(iter(train_loader))

# from torch.nn.utils import clip_grad_value_
# from transformers import BitsAndBytesConfig
# from peft import get_peft_model

# nf4_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_compute_dtype=torch.bfloat16
# )
# base_class = AutoModelForCausalLM if pred_type == 'LM' else AutoModelForSequenceClassification
# base_model = base_class.from_pretrained(TARGET_MODEL,quantization_config=nf4_config, \
#                                                           device_map={"":0},use_flash_attention_2=True)
# peft_config = config_class(**config_kwargs)    
# model = get_peft_model(base_model, peft_config)
# model.config.pad_token_id = tokenizer.pad_token_id

# model.load_state_dict(torch.load(save_model_path+'/weights.pth'),strict=False)
# model = model.half()
# if pred_type == 'LM':
#     model_lm = LM(model,tokenizer,config_kwargs.get('num_virtual_tokens', 0),alpha,config_type)
# else:
#     model_lm = Classification(model,alpha)

# yhat = []
# for input_ids,attention_mask in val_loader:
#     input_ids,attention_mask = input_ids.to('cuda'),attention_mask.to('cuda')
#     out = model_lm.predict(input_ids,attention_mask).detach().cpu().numpy()
#     yhat.append(out)
# yhat = np.concatenate(yhat)
# auc = roc_auc_score(val.label.to_numpy(), yhat)
# print(auc)